In [1]:
import config
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from Utils.hdf5DatasetWriter import HDF5DatasetWriter

from imutils import paths
import numpy as np
import json
import cv2
import os

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
trainPaths = list(paths.list_images(config.TRAIN_IMAGES))
trainLabels = [p.split(os.path.sep)[-3] for p in trainPaths]
le = LabelEncoder()
trainLabels = le.fit_transform(trainLabels)

trainPaths, testPaths, trainLabels, testLabels = train_test_split(trainPaths, trainLabels, 
                                                                  test_size=config.NUM_TEST_IMAGE, stratify=trainLabels,
                                                                  random_state=42)

M = open(config.VAL_MAPPING).read().strip().split('\n')
M = [r.split('\t')[:2] for r in M]
valPaths = [os.path.sep.join([config.VAL_IMAGES, m[0]]) for m in M]
valLabels = le.fit_transform([m[1] for m in M])

datasets = [('train', trainPaths, trainLabels, config.TRAIN_HDF5),
            ('test', testPaths, testLabels, config.TEST_HDF5),
            ('val', valPaths, valLabels, config.VAL_HDF5)]

In [4]:
(R, G, B) = ([], [], [])

for dType, paths, labels, path in datasets:
    print('[INFO] Building {}'.format(dType))
    writer = HDF5DatasetWriter((len(paths), 64, 64, 3), path)
    
    for (i, (imagePath, label)) in enumerate(zip(paths, labels)):
        image = cv2.imread(imagePath)
        if dType == 'train':
            b, g, r = cv2.mean(image)[:3]
            R.append(r)
            G.append(g)
            B.append(b)
        writer.add([image], [label])
        if i%1000 == 0:
            print(i)
    writer.close()
print('[INFO] Write json file')
D = {'R':np.mean(R), 'G':np.mean(G), 'B':np.mean(B)}
f = open(config.DATASET_MEAN, 'w')
f.write(json.dumps(D))
f.close()

[INFO] Building train
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
[INFO] Building test
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
[INFO] Building val
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
[INFO] Write json file
